In [26]:
import hashlib
from Cryptodome.Cipher import AES
from Cryptodome.Util.Padding import pad
import simplejson as json
from decimal import Decimal
from collections import OrderedDict
import requests

In [ ]:
EXCHANGE_SECRET = "f430c1f3a9c9484e86c8aefc587753ee7db6735ffd77f6a48705dbdce70a755b" # hardcoded
STEAM_ID = "76561198219482843" # Steam ID

In [ ]:
url = "https://services.synthriders.net/api/game/auth/exchange"

# are all headers required?
headers = {
    "User-Agent": "UnityPlayer/2019.4.40f1 (UnityWebRequest/1.0, libcurl/7.80.0-DEV)",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded",
    "X-Unity-Version": "2019.4.40f1"
}

data = {
    "authType": "0",
    "playerId": STEAM_ID,
    "nonce": "14000000b518e25e334efc68db56730f01001001c161ea671800000001000000020000005d858eaa98901db7164c600009000000b20000003200000004000000db56730f0100100108810d00c08ff14f8d02a8c000000000f9fedf6779aefb670100d54a0400000000008b4c49f1bb999e86d775b87f7908f9e1671b6997343dda9b97e6776c74d70b98194af09766660e15d0968a6250e88e57aac59b1602ad65e73924ab206c7cc01e2b2dd33bd000e8c33c69fdd97f436248fb38b3edf90a2cd1cc867c46807b4a1daa1bd6c3b3974ef7cb3346e307a018166f2f2a516913619086248c8730cba6c5", # get this via https://github.com/philippj/SteamworksPy
    "friends": "",
    "name": "henrydatei"
}

session = requests.Session()

response = session.post(url, headers=headers, data=data)
print("Response status:", response.status_code)
print("Response content:", response.text)
print("Cookies stored in session:", session.cookies.get_dict())

# Return should be something like
# Set-Cookie: connect.sid=s%3A7NsufH3zRuzZzY6at1CYB7fVnBT59bC-.E%2BqVIDiIZJF%2Fab3egRk%2BmsUouVpBRycNvDjrjOS01rU; Path=/; HttpOnly

# {
#   "id": 411627,
#   "key": "9f9a9f741299f8ac",
#   "cookie": "7NsufH3zRuzZzY6at1CYB7fVnBT59bC-",
#   "multiplayerBanned": 0,
#   "blockedList": []
# }

AUTHENTICATION_KEY = response.json()["key"]
print("AUTHENTICATION_KEY:", AUTHENTICATION_KEY)

Response status: 500
Response content: {
  "error": {
    "message": "Failed to authenticate Steam ticket"
  }
}
Cookies stored in session: {}


In [ ]:
salted_string = f"{EXCHANGE_SECRET}-{AUTHENTICATION_KEY}-{STEAM_ID}-{EXCHANGE_SECRET}"
SALTED_KEY = hashlib.md5(salted_string.encode('utf-8')).hexdigest()

print("SALTED_KEY:", SALTED_KEY) # Logs: 30ab335b65522cb7e056dce85bc0e509

SALTED_KEY: 30ab335b65522cb7e056dce85bc0e509


In [ ]:
def encrypt(data: str, key: str) -> str:
    key_bytes = key.encode('utf-8')
    iv_bytes = key[:16].upper().encode('utf-8')
    cipher = AES.new(key_bytes, AES.MODE_CBC, iv_bytes)
    data_bytes = data.encode('utf-8')
    padded_data = pad(data_bytes, AES.block_size, style='pkcs7')
    encrypted_bytes = cipher.encrypt(padded_data)
    return encrypted_bytes.hex().upper()

In [ ]:
data_dict = OrderedDict([
    ("leaderboardInfo", OrderedDict([
        ("songHash", "d652be1a31c549d06fa15c11c9b793c193c5c2c6094192e4c41418878b5c99a9"),
        ("songName", "Watch Your Tongue"),
        ("songSubName", ""),
        ("songAuthorName", "PiSk, Little Violet"),
        ("levelAuthorName", ""),
        ("bpm", 0),
        ("difficulty", 4),
        ("mode", 0),
        ("maxScore", 0),
        ("maxCombo", 601),
        ("challenge", 0)
    ])),
    ("scoreInfo", OrderedDict([
        ("baseScore", 1383210),
        ("notesHit", 595),
        ("modifiers", 0),
        # Use Decimal with the exact string value to force precise formatting.
        ("goodHitPercent", Decimal("0.55574041604995728")),
        ("poorHitPercent", Decimal("0.21963393688201904")),
        ("perfectHitPercent", Decimal("0.21464225649833679")),
        # For 0.0 you can use Decimal as well if needed.
        ("punchIntensityAvg", Decimal("0.0")),
        ("maxCombo", 171),
        ("hmd", 8)
    ]))
])

json_text = json.dumps(data_dict, separators=(',',':'), use_decimal=True)
print("JSON text:")
print(json_text)

encrypted_data = encrypt(json_text, SALTED_KEY)
print("\nEncrypted Data:")
print(encrypted_data) # Logs: 7399E1F499538AD319707CD313DBBF3D6C8C17F1215546F688F4DBC56DAF7DB7BC75C6D346DA1350852554AF22E2705FBF4336759A7F5304140E9E37670E159E69FEF889E3BA603467B9290FC5B588D188B79C1E92926F77009A45F243D7879615D6F189B8FF6DC4CEB52F3F64C77559F527CE25706D4B3A0BAB68A8CE8D654C1368C2BE8DDA9ED03C154B35B86AD5550CCA5DDB1D28FCF99C1DA46E12C3BD6E7E0F0995E0EB69831577B5B8AD3138B204B6E0A3466892B49AAFE3EC6A9B3C475734ADF29EDCCB225E4D9C9BC2577730086565A6FC2FDC05D51DE7D238702AA59EAF7B54296181A0B7D75A35DFEF4563427C819AAA68001B79CFA429A497CDAD9EDD7456E70450D26206F57CCF7570B141B87C9CF643F6D684E2599745104288D4D100847B6094426954F93E3486AC6EA082C5055A61DAE191DF182AD03F6EF24E99CEA511CC49B926231573F3CF37037822901D01711A9CAF4DDA9425FD657DF02555F70A682E61EF86853D03C9573ABFE8EC8F9E739523CDC8AD66FD3434BFA741FBB84819A7175DDF33791D24865A5D832487551398D03530050A3E572337A53C22F395B7C16DD9CEFA03B91988B7CE4B3F40CB33434E761FAC49980C782B54D498CC6FDCD24CAE9C42F0F78824B049A52D5875ADAF17EE833D404E631656AF275CB7DFC31516AF804307938B0E20D455E929D2CB02A2A89C9F7B38EC9B98

JSON text:
{"leaderboardInfo":{"songHash":"d652be1a31c549d06fa15c11c9b793c193c5c2c6094192e4c41418878b5c99a9","songName":"Watch Your Tongue","songSubName":"","songAuthorName":"PiSk, Little Violet","levelAuthorName":"","bpm":0,"difficulty":4,"mode":0,"maxScore":0,"maxCombo":601,"challenge":0},"scoreInfo":{"baseScore":1383210,"notesHit":595,"modifiers":0,"goodHitPercent":0.55574041604995728,"poorHitPercent":0.21963393688201904,"perfectHitPercent":0.21464225649833679,"punchIntensityAvg":0.0,"maxCombo":171,"hmd":8}}

Encrypted Data:
7399E1F499538AD319707CD313DBBF3D6C8C17F1215546F688F4DBC56DAF7DB7BC75C6D346DA1350852554AF22E2705FBF4336759A7F5304140E9E37670E159E69FEF889E3BA603467B9290FC5B588D188B79C1E92926F77009A45F243D7879615D6F189B8FF6DC4CEB52F3F64C77559F527CE25706D4B3A0BAB68A8CE8D654C1368C2BE8DDA9ED03C154B35B86AD5550CCA5DDB1D28FCF99C1DA46E12C3BD6E7E0F0995E0EB69831577B5B8AD3138B204B6E0A3466892B49AAFE3EC6A9B3C475734ADF29EDCCB225E4D9C9BC2577730086565A6FC2FDC05D51DE7D238702AA59EAF7B54296181A0B7

In [ ]:
upload_url = "https://services.synthriders.net/api/game/upload"

# are all headers required?
headers = {
    "User-Agent": "UnityPlayer/2019.4.40f1 (UnityWebRequest/1.0, libcurl/7.80.0-DEV)",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Content-Type": "application/x-www-form-urlencoded",
    "X-Unity-Version": "2019.4.40f1"
}
form_data = {
    "data": encrypted_data
}

response = session.post(upload_url, headers=headers, data=form_data)
print("Upload status code:", response.status_code)
print("Upload response content:", response.text)